In [318]:
import argparse
import os
import random
import shutil
import time
import warnings
import datetime
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F




os.chdir('/EdMIPS')
# EdMIPS/models ディレクトリへのパスを追加
sys.path.append('/EdMIPS/models')

from matplotlib.colors import ListedColormap
from pprint import pprint
import matplotlib.ticker as ticker
from models.quant_efficientnet import BasicCNNBlock
import models as models
# カレントディレクトリを 'EdMIPS' に変更
os.chdir('/EdMIPS')

device_count = torch.cuda.device_count()
print('Number of devices: {}'.format(device_count))
# setting device
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

Number of devices: 2


In [319]:
def validate(val_loader, model, criterion):
    acc1_avg = 0
    acc5_avg = 0
    # switch to evaluate mode
    model.eval()
    model.to(device)

    with torch.no_grad():

        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)
            # target = target.to(device).view(-1)
            
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            acc1_avg += float(acc1)
            acc5_avg += float(acc5)
            # check
            # print(f'{i}  acc1:{int(acc1)}  acc5:{int(acc5)}')
        # TODO: this should also be done with the ProgressMeter
        # print(' * Acc@1 {top1:.3f} Acc@5 {top5:.3f}'
        #       .format(top1=top1, top5=top5))
    acc1_avg = acc1_avg / len(val_loader)
    acc5_avg = acc5_avg / len(val_loader)
    return acc1_avg,acc5_avg

In [320]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            #correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
    
def adjust_learning_rate(optimizer, epoch, config):
    """Sets the learning rate to the initial LR decayed by 10 every step_epochs"""
    step_epoch = 10
    lr = config['lr'] * (0.1 ** (epoch // step_epoch))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [321]:
config = {
    'model_name': 'quanteffnet_cfg_2468',
    'arch_path' : '/EdMIPS/main_model_output/efficient_b0_w2468a2468_100_csd0.01_forlossynet/model_best.pth.tar',
    'batch_size': 10,
    'workers'   : 16,
    'epoch'     : 25,
    'momentum'  : 0.9,
    'weight-decay':1e-4,
    'lr'        : 0.1
}
DATASETDIR ='/datasets/imagenet-100'

In [322]:
# Data loading code
traindir = os.path.join(DATASETDIR, 'train')
valdir = os.path.join(DATASETDIR , 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

# if 'efficientnet' in args.arch:
#     crop_size, short_size = model.get_image_size(args.arch)
    
# if 'inception' in model.:
#     crop_size, short_size = 299, 342
# elif "effnet" in model:  # 要修正
#     crop_size, short_size = model.res, model.res
#     # model.resとは？
# else:
crop_size, short_size = 224, 256


train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
print(crop_size, short_size)
train_sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=config['batch_size'], shuffle=(train_sampler is None),
    num_workers=config['workers'], pin_memory=True, 
    sampler=train_sampler)

val_dataset =datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.Resize(short_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize,
    ]))
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config['batch_size'], shuffle=False,
    num_workers=config['workers'], pin_memory=True)

224 256


In [323]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.SGD(model.parameters(), config['lr'],
                                momentum=config['momentum'],
                                weight_decay=config['weight-decay'])

cudnn.benchmark = True

In [324]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))


mixefnet_dir='/EdMIPS/arch_output/mixeffnet_b0_w2468a2468_100_csd0.01_forlossynet/arch_model_best.pth.tar'
model = models.__dict__[config["model_name"]](mixefnet_dir)

checkpoint = torch.load(config['arch_path'])
start_epoch = checkpoint['epoch']
print(start_epoch)
best_acc1 = checkpoint['best_acc1']
print(best_acc1)
best_acc1 = best_acc1.to(device)
print(best_acc1)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])


archas: [4, 4, 6, 6, 4, 4, 6, 6, 6, 6, 2, 6, 6, 2, 6, 4, 4, 4, 4, 4, 2, 6, 6, 2, 4, 4, 2, 6, 4, 4, 2, 6, 6, 6, 4, 2, 6, 6, 4, 4, 2, 4, 6, 4, 4, 2, 6, 6, 4, 2, 2, 6, 6, 4, 2, 4, 2, 6, 4, 4, 2, 4, 4, 4, 4, 2, 4, 2, 4, 4, 2, 4, 2, 4, 2, 4, 6, 6, 4, 6]
archws: [6, 4, 2, 6, 2, 2, 8, 6, 8, 2, 2, 8, 2, 2, 4, 2, 8, 8, 8, 2, 2, 8, 4, 2, 4, 2, 8, 8, 8, 2, 2, 6, 4, 2, 2, 2, 4, 4, 2, 6, 2, 8, 8, 8, 2, 2, 6, 4, 2, 2, 2, 8, 4, 2, 2, 2, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2, 8, 8, 4, 6]
99
tensor(77.7400, device='cuda:0')
tensor(77.7400, device='cuda:0')


In [325]:
print(model)

EfficientNet(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (features): Sequential(
    (0): BasicCNNBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): QuantActivConv2d(
            (activ): HWGQ()
            (conv): QuantConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          )
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): QuantActivConv2d(
              (activ): HWGQ()
              (conv): QuantConv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
     

In [331]:

# arch_outputはa,wのビット数が保持されてるけど、main_model_outputではa,wのビット数は保持されない？
# model,_ = get_model(config)
# print(model)
head_model = nn.Sequential(*list(model.features[:7]))
edge_model = nn.Sequential(*list(model.features[7:]))


class LossyNetModel(nn.Module):
    def __init__(self, head_model, edge_model):
        super(LossyNetModel, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head_model =head_model
        self.edge_model = edge_model
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features=1280, out_features=100, bias=True)
        )
    def forward(self, x):
        
        x = self.head_model(x)
        # x = torch.nn.functional.dropout(x,p=0.2,training=True)
        x = self.edge_model(x)
        x = self.pool(x)
        x = self.classifier(x.view(x.shape[0], -1))
        return x
    

new_model =LossyNetModel(head_model,edge_model).to(device)
# setting device
print(new_model)



LossyNetModel(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (head_model): Sequential(
    (0): BasicCNNBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): QuantActivConv2d(
            (activ): HWGQ()
            (conv): QuantConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          )
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): QuantActivConv2d(
              (activ): HWGQ()
              (conv): QuantConv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
  

In [332]:
# 既存モデルを分割して再度くっつけたモデルの精度確認(dropoutなし)
new_model.to(device)
acc1,acc5 = validate(val_loader,new_model,criterion)
print(f"acc1:{acc1}%")
print(f"acc5:{acc5}%")

acc1:1.04%
acc5:5.46%


In [329]:
# 既存モデルの精度確認
model.to(device)
acc1,acc5 = validate(val_loader,model,criterion)
print(f"acc1:{acc1}%")
print(f"acc5:{acc5}%")

acc1:79.1%
acc5:94.34%
